In [1]:
#importing libraries

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#send request

url=['https://www.imdb.com/list/ls068082370/','https://www.imdb.com/list/ls068082370/?sort=list_order,asc&st_dt=&mode=detail&page=2','https://www.imdb.com/list/ls068082370/?sort=list_order,asc&st_dt=&mode=detail&page=3']
page=[]
soup=[]

for i in url:
    page.append(requests.get(i))

for i in page:
    soup.append(BeautifulSoup(i.text,'html.parser'))

In [3]:
#read id

element=[]
movie_id=[]
for i in soup:
    element.append(i.find_all(class_='lister-item-index unbold text-primary'))
for i in element:
    for item in i:
        movie_id.append(item.text)
for i in range(250):
    movie_id[i]=movie_id[i][:-1]

In [4]:
#read movie name

element=[]
movie=[]

for i in soup:
    element.append(i.find_all(class_='lister-item-content'))
for i in element:
    for item in i:
        movie.append(item.a.text)

In [5]:
#read year of release

year=[]
element=[]
for i in soup:
    element.append(i.find_all(class_='lister-item-year text-muted unbold'))
for i in element:
    for item in i:
        year.append(item.text)
for i in range(250):
    year[i]=year[i].replace('(','')
    year[i]=year[i].replace(')','')

In [6]:
#read genre

genre=[]
element=[]
for i in soup:
    element.append(i.find_all(class_='genre'))
for i in element:
    for item in i:
        genre.append(item.text)
for i in range(250):
    genre[i]=genre[i][1:-12]

In [7]:
#read IMDB rating

r=[]
rating=[]
element=[]
for i in soup:
    element.append(i.find_all(class_='ipl-rating-star__rating'))
for i in element:
    for item in i:
        r.append(item.text)
c=0
for i in range(5750):
    if i==c:
        rating.append(r[i])
        c=c+23

In [8]:
#read metascore

metascore=[]
element=[]
for i in soup:
    element.append(i.find_all('div', class_='inline-block ratings-metascore'))
for i in element:
    for item in i:
        metascore.append(item.text)
for i in range(203):
    metascore[i]=metascore[i].replace('        \n        Metascore\n            ','')
    metascore[i]=metascore[i].replace('\n','')

In [9]:
#read about

about1=[]
element=[]
for i in soup:
    element.append(i.find_all('p'))#, class_='text-muted'))
for i in element:
    for item in i:
        about1.append(item.text)
about1=about1[13:]
about=[]
c=1
for i in range(1000):
    if i==c:
        about.append(about1[i])
        c=c+4
for i in range(250):
    about[i]=about[i].replace('\n    ','')

In [10]:
#read directors and actors

element=[]
k=[]
director=[]
actor=[]
d=[]
a=[]
for i in soup:
    element.append(i.find_all('p', class_='text-muted text-small'))
c=0
for i in element:
    for item in i:
        if c%3==1:
            k.append(item.text)
        c=c+1
for i in range(250):
    k[i]=k[i].replace('\n','')
    d.append(k[i].split('|')[0])
    a.append(k[i].split('|')[1])
for i in range(250):
    d[i]=d[i].replace('Director:','')
    d[i]=d[i].replace('Directors:','')
    director.append(d[i])
    a[i]=a[i].replace('Star:','')
    a[i]=a[i].replace('Stars:','')
    actor.append(a[i])
for i in range(250):
    director[i]=director[i].replace('    ','')
    actor[i]=actor[i].replace('     ','')

In [11]:
#read number of votes and gross income

k1=[]
element=[]
for i in soup:
    element.append(i.find_all('p'))
for i in element:
    for item in i:
        k1.append(item.text)
k1=k1[13:]
k=[]
c=3
for i in range(1000):
    if i==c:
        k.append(k1[i])
        c=c+4
for i in range(250):
    k[i]=k[i].replace('\n','')
    k[i]=k[i].replace('Votes:','')
    k[i]=k[i].replace('Gross:','')
votes=[]
gross=[]
for i in range(250):
    l=k[i].split('| ')
    if len(l)==1:
        votes.append(l[0])
        gross.append(np.NaN)
    else:
        votes.append(l[0])
        gross.append(l[1])

In [12]:
#creating dataframe

final_array=[]

for id, name, rel_year, gen, rat, ab, direc, act, vote, gros in zip(movie_id, movie, year, genre, rating, about, director, actor, votes, gross):
    final_array.append({'Movie_ID':id, 'Movie_Title':name, 'Year_of_Release':rel_year, 'Genre':gen, 'IMDb_Ratings':rat, 'About':ab, 'Director':direc, 'Stars':act, 'Number_of_Votes': vote, 'Gross_Income':gros})
    
data=pd.DataFrame(final_array)
data=data[['Movie_ID','Movie_Title','Year_of_Release','Genre','IMDb_Ratings','About','Director','Stars','Number_of_Votes','Gross_Income']]

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
Movie_ID           250 non-null object
Movie_Title        250 non-null object
Year_of_Release    250 non-null object
Genre              250 non-null object
IMDb_Ratings       250 non-null object
About              250 non-null object
Director           250 non-null object
Stars              250 non-null object
Number_of_Votes    250 non-null object
Gross_Income       133 non-null object
dtypes: object(10)
memory usage: 19.6+ KB


In [17]:
#saving the dataframe

data.to_csv(r'C:\Users\Malo\Top-250-Movies-of-all-time\data.csv', header=False, index=False)